# 11. Идея обучения НС градиентным алгоритмом

# 12. Алгоритм back propagation. Пример работы

In [2]:
import torch
import numpy as np
from random import randint
import matplotlib.pyplot as plt

In [3]:
def act(z):
    return torch.tanh(z)

In [6]:
def df(z):
    s = act(z)
    return 1 - s * s

In [16]:
def go_forward(x_inp, w1, w2):
    z1 = torch.mv(w1[:, :3], x_inp) + w1[:, 3]
    s = act(z1)

    z2 = torch.dot(w2[:2], s) + w2[2]
    y = act(z2)
    return y, z1, z2

In [17]:
torch.manual_seed(1)

W1 = torch.rand(8).view(2, 4) - 0.5
W2 = torch.rand(3) - 0.5

# обучающая выборка (она же полная выборка)
x_train = torch.FloatTensor([(-1, -1, -1), (-1, -1, 1), (-1, 1, -1), (-1, 1, 1),
                            (1, -1, -1), (1, -1, 1), (1, 1, -1), (1, 1, 1)])
y_train = torch.FloatTensor([-1, 1, -1, 1, -1, 1, -1, -1])

In [20]:
lmd = 0.05              # шаг обучения
N = 5000                # число итераций обучения
total = len(y_train)    # размер обучающей выборки

In [21]:
for _ in range(N):
    k = randint(0, total-1)
    x = x_train[k] # случайный выбор образа из обучающей выборки
    y, z1, out = go_forward(x, W1, W2) # прямой проход по НС и вычисление выходных значений нейронов
    e = y - y_train[k]  # производная квадратиечской функции потерь
    delta = e * df(out) #вычисление локального градиента
    delta2 = W2[:2] * delta * df(z1) #вектор из 2-х локальных градиентов скрытого слоя
    
    W2[:2] = W2[:2] - lmd * delta * z1 #корректировка весов связей последнего слоя
    W2[2] = W2[2] - lmd * delta #корректировка bias
    
    #корректировка связей первого слоя
    W1[0, :3] = W1[0, :3] - lmd * delta2[0] * x
    W1[1, :3] = W1[1, :3] - lmd * delta2[1] * x

In [22]:
# тестирование обученной НС
for x, d in zip(x_train, y_train):
    y, z1, out = go_forward(x, W1, W2)
    print(f"Выходное значение НС: {y} => {d}")

# результирующие весовые коэффициенты
print(W1)
print(W2)

Выходное значение НС: -0.9564250707626343 => -1.0
Выходное значение НС: 0.9945653676986694 => 1.0
Выходное значение НС: -0.9997304677963257 => -1.0
Выходное значение НС: 0.9694713354110718 => 1.0
Выходное значение НС: -0.9997363090515137 => -1.0
Выходное значение НС: 0.969562828540802 => 1.0
Выходное значение НС: -0.999871551990509 => -1.0
Выходное значение НС: -0.9483084082603455 => -1.0
tensor([[ 0.4705,  0.4024, -0.9522,  0.2347],
        [ 0.6854,  0.7586, -1.4179,  0.2544]])
tensor([-1.4237, -2.5644, -0.8941])
